# 데이터 전처리
AI-Hub에서 제공한 문서 요약 데이터에서 이메일, 기자이름, 출판사 이름, 특수문자 등을 제거하고 기존 데이터와 전처리한 데이터, KoBART 예제에서 제공한 데이터를 추가하여 KoBART 모델 학습을 위한 데이터셋을
구축해보자.

In [ ]:
import pandas as pd
import numpy as np
import json
import re
from tqdm import tqdm
from pandas import json_normalize
import torch
from transformers import PreTrainedTokenizerFast
from transformers.models.bart import BartForConditionalGeneration

In [ ]:
DATA_PATH = '.......'

In [250]:
with open(f'{DATA_PATH}train_original_news.json') as f:
    news_data_train = json.load(f)

In [61]:
with open(f'{DATA_PATH}valid_original_news.json') as f:
    news_data_val = json.load(f)

In [62]:
with open(f'{DATA_PATH}train_original_personal.json') as f:
    personal_data_train = json.load(f)

In [63]:
with open(f'{DATA_PATH}valid_original_personal.json') as f:
    personal_data_val = json.load(f)

In [253]:
# Json 형식의 데이터에서 필요한 데이터만 추출하는 함수
def make_df(data):
    l1 = []
    for i in tqdm(range(len(data['documents']))):
        news_id = data['documents'][i]['id']
        category = data['documents'][i]['category']
        date = data['documents'][i]['publish_date']
        title = data['documents'][i]['title']
        string = ''
        string2 = ''
        extractive = data['documents'][i]['extractive']
        abstractive = data['documents'][i]['abstractive'][0]
        for j in range(len(data['documents'][i]['text'])):
            for k in range(len(data['documents'][i]['text'][j])):
                    string += data['documents'][i]['text'][j][k]['sentence']
                    if data['documents'][i]['text'][j][k]['index'] in extractive:
                        string2 += data['documents'][i]['text'][j][k]['sentence']
        l1.append([news_id, category, date, title, string, string2, abstractive])
    return l1

In [139]:
news_train = make_df(news_data_train)
news_train = pd.DataFrame(news_train, columns = ['id', 'category', 'date', 'title', 'sentence', 'extractive', 'abstractive'])
news_train.head()

100%|██████████| 243983/243983 [00:02<00:00, 96418.22it/s] 


,id,category,date,title,sentence,extractive,abstractive
0,290741778,종합,2018-01-05 18:54:55,"논 타작물 재배, 2월 말까지 신청하세요",ha당 조사료 400만원…작물별 차등 지원이성훈 sinawi@hanmail.net전...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...
1,290741792,종합,2018-01-05 18:53:20,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성","8억 투입, 고소천사벽화·자산마을에 색채 입혀이성훈 sinawi@hanmail.ne...",여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...,여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...
2,290741793,스포츠,2018-01-05 18:52:15,“새해 정기 받고 올해는 반드시 일내자!”,전남드래곤즈 해맞이 다짐…선수 영입 활발이성훈 sinawi@hanmail.net전남...,임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목표를...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...
3,290741794,정치,2018-01-05 18:50:17,"농업인 역량 강화, 새해 실용교육 실시","11~24일, 매실·감·참다래 등 지역특화작목이성훈 sinawi@hanmail.ne...",광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 1050...,"광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작..."
4,290741797,종합,2018-01-05 18:52:36,타이완 크루즈관광객 4천여명‘전남’온다,"홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항이성훈 sinawi@hanmai...",전라남도는 올해 4월과 6월 홍콩 크루즈선사 스타크루즈의 5만톤급‘아쿠아리우스’호가...,올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...


In [140]:
news_val = make_df(news_data_val)
news_val = pd.DataFrame(news_val, columns = ['id', 'category', 'date', 'title', 'sentence', 'extractive', 'abstractive'])
news_val.head()

100%|██████████| 30122/30122 [00:00<00:00, 100373.26it/s]


,id,category,date,title,sentence,extractive,abstractive
0,340626877,정치,2019-04-08 17:43:00,"文대통령 ""5G는 4차산업혁명 시대의 고속도로""","[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, 체...","[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, 체...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...
1,340626896,종합,2019-04-08 17:41:00,"""손학규 물러나라""...바른계, 최고위 회의 '보이콧'",] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.바른미래당이 8일 연 최...,바른미래당이 8일 연 최고위원 회의에는 7명의 최고위원 중 손학규 대표와 김관영 원...,8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...
2,340626904,"IT,과학",2019-04-08 17:44:00,"""마치 한 장소처럼"" ... 5G로 서울·부산·광주 원격협연",[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀.지난 3일 한국이 세계 최초로 5...,지난 3일 한국이 세계 최초로 5세대(5G) 이동통신 서비스를 상용화한 것을 기념하...,지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...
3,340627450,사회,2019-04-08 17:48:00,"""이웃도 모르는데, 이게 서울 미래냐""...'고층 재개발 요구' 작심 비판한 박원순",] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리...,] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...
4,340627465,경제,2019-04-08 17:49:00,"""SK·알파벳 벤치마킹해야""...KB운용, KMH에 주주서한","[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를 ...",기업 지배구조 개선 등을 통해 높은 수익률을 올리는 것을 목표로 하는 공모펀드 'K...,주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...


In [143]:
personal_train = make_df(personal_data_train)
personal_train = pd.DataFrame(personal_train, columns = ['id', 'category', 'date', 'title', 'sentence', 'extractive', 'abstractive'])
personal_train.head()

100%|██████████| 56760/56760 [00:00<00:00, 80037.97it/s]


,id,category,date,title,sentence,extractive,abstractive
0,100062073,오피니언,2011-09-01 00:03:01,[사설] 기부문화 확산 은근히 강조한 李대통령,"이명박 대통령이 어제 30대 그룹 총수를 모아놓고 ""시대적 요구는 역시 총수가 앞장...","이명박 대통령이 어제 30대 그룹 총수를 모아놓고 ""시대적 요구는 역시 총수가 앞장...",이명박 대통령은 어제 30대 그룹 총수를 모아놓고 시대적 요구는 역시 총수가 앞장서...
1,100062076,오피니언,2011-09-01 00:01:01,[사설] `류우익 對北전략` 지킬것과 바꿀것 확실히해야,이명박 정부의 첫 대통령실장을 지낸 류우익 주중대사가 통일부 장관에 내정되면서 대북...,이명박 정부의 첫 대통령실장을 지낸 류우익 주중대사가 통일부 장관에 내정되면서 대북...,류우억 주중대사가 통일부 장관에 내정되면서 대북정책에 대하여 일관성 있게 지켜야...
2,100062209,증권,2011-09-01 00:04:00,"[사설] 한은법 개정, 이젠 금융위기 대응력 높일 때",한국은행법 개정안이 우여곡절 끝에 국회 본회의를 통과했다.이 법안은 글로벌 금융위기...,한국은행법 개정안이 우여곡절 끝에 국회 본회의를 통과했다.이 법안은 글로벌 금융위기...,한국은행은 금융 위기 이후 금융 시스템 안정을 위해 권한이 더욱 부여된 개정안을 통...
3,100062217,증권,2011-09-01 00:03:00,[사설] 기부문화 확산 은근히 강조한 李대통령,"이명박 대통령이 어제 30대 그룹 총수를 모아놓고 ""시대적 요구는 역시 총수가 앞장...","이명박 대통령이 어제 30대 그룹 총수를 모아놓고 ""시대적 요구는 역시 총수가 앞장...",30대 그룹 총수들을 모아 회동을 가진 이명박 대통령은 간접적으로 기부에 대해 언급...
4,100062218,증권,2011-09-01 00:02:00,[사설] 포퓰리즘식 비정규직 보호 得보다 失이 크다,한나라당과 정부가 비정규직 대책을 이르면 다음주 중 내놓을 예정이다.비정규직 임금을...,그러나 좋은 취지에도 불구하고 재계의 능력을 벗어나는 정치권의 강요는 고용 자체를 ...,내년 총선ㆍ대선을 앞두고 한나라당의 비정규직을 대상으로 한 포퓰리즘 대책들은 정치권...


In [144]:
personal_val = make_df(personal_data_val)
personal_val = pd.DataFrame(personal_val, columns = ['id', 'category', 'date', 'title', 'sentence', 'extractive', 'abstractive'])
personal_val.head()

100%|██████████| 7008/7008 [00:00<00:00, 78078.34it/s]


,id,category,date,title,sentence,extractive,abstractive
0,365247698,종합,2019-10-31 04:40:00,"[사설] 이해찬 대표 ‘조국 사태’ 사과, 과감한 인적 쇄신으로 이어져야",더불어민주당 이해찬 대표가 30 일 오후 국회에서 기자간담회를 열고 조국 전 법무부...,"더불어민주당 이해찬 대표가 30 일 기자간담회를 열고 '조국 사태'와 관련, ""국민...",이해찬 대표가 조국 사태와 관련 송구한 입장 표명이 과감한 인적 쇄신으로 이어져야 한다.
1,365247823,정치,2019-10-31 05:00:00,"[사설] ""전기료 인상 절대 없다""던 약속 누가 책임지나",탈원전 정책에서 비롯된 한국전력의 경영 부담이 결국 전기료 인상을 초래하게 됐다.김...,"탈원전 정책에서 비롯된 한국전력의 경영 부담이 결국 전기료 인상을 초래하게 됐다.""...",탈원전 정책에서 비롯된 한국전력의 경영 악화가 결국 전기료 인상이라는 결과를 낳아 ...
2,365247824,정치,2019-10-31 05:00:00,[사설] 언제까지 북한 억지에 끌려다닐 건가,북한이 그제 금강산국제관광국 명의로 통일부와 현대아산에 통지문을 보내 금강산관광 시...,북한이 그제 금강산국제관광국 명의로 통일부와 현대아산에 통지문을 보내 금강산관광 시...,북한이 통일부와 현대아산에 금강산 관광 시설 철거와 관련하여 문서 교환 방식의 협의...
3,365247941,종합,2019-10-31 05:00:00,"[사설] 물가상승 OECD 최저, 뚜렷해진 디플레 조짐",지난달 한국의 소비자물가상승률이 경제협력개발기구(OECD) 회원국 중 가장 낮았다....,OECD가 집계한 국가별 소비자물가 통계에서 9 월 한국 상승률은 전년 동월 대비 ...,OECD가 집계한 소비자물가 통게에서 9월 한국 상승률은 전년 동월 대비 -0.4%...
4,365250721,종합,2019-10-31 06:30:00,"[사설]'탈원전 재앙', 결국 국민 전기료 부담만 늘린다",김종갑 한국전력 사장이 1 조1 천억원이 넘는 각종 전기료 특례 할인을 모두 폐지하...,김종갑 한국전력 사장이 1 조1 천억원이 넘는 각종 전기료 특례 할인을 모두 폐지하...,김종갑 한국전력 사장이 1조 1천억원 규모의 전기료 특례 할인 폐지 및 전기요금 원...


In [255]:
news_train.to_feather(f'{DATA_PATH}news_train.ftr')
news_val.to_feather(f'{DATA_PATH}news_val.ftr')
personal_train.to_feather(f'{DATA_PATH}personal_train.ftr')
personal_val.to_feather(f'{DATA_PATH}personal_val.ftr')

In [ ]:
news_train = pd.read_feather(f'{DATA_PATH}news_train.ftr')
personal_train = pd.read_feather(f'{DATA_PATH}personal_train.ftr')
news_val = pd.read_feather(f'{DATA_PATH}news_val.ftr')
personal_val = pd.read_feather(f'{DATA_PATH}personal_val.ftr')

In [4]:
# 이메일 제거 함수
def remove_email(sentence):
    if '.net' in sentence:
        return sentence.split('.net')[1]

    elif 'co.kr' in sentence:
        return sentence.split('co.kr')[1]
        
    elif '.com' in sentence:
        return sentence.split('.com')[1]
    
    else:
        return sentence
    
def remove_email2(sentence):
    #일단, net, co.kr, com으로 다 지우고
    #풀형식으로 되어있는 email 제거
    #풀로 아니고 이상한 이메일도 제거
    email_pattern1 = re.compile('[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+|[a-zA-Z0-9+-]+@')
    email = email_pattern1.findall(sentence)
    for i in email:
        sentence =  sentence.replace(i,'')
    return sentence

In [159]:
#기자나 ] 없애기
def remove_reporter(sentence):
    reporter_pattern = re.compile(r"[ *[가-힣]* [기자|팀]+ *.*] |] |\([가-힣]+=[가-힣]* 기자\)|\<[가-힣]+=[가-힣]* 기자\>|/특별취재팀=*[가-힣]+ 기자특별취재팀=*[가-힣]+ 기자")
    reporter = reporter_pattern.findall(sentence)
    for i in reporter:
        sentence = sentence.replace(i,'')
    return sentence 

In [6]:
#~제공없애기
def remove_provider(sentence):
    provider_pattern = re.compile(r"[[a-zA-Z]*[가-힣]* 제공 *]|\([a-zA-Z]*[가-힣]* 제공 *\)|\<[a-zA-Z]*[가-힣]* 제공 *\>") # ~~기자
    provider = provider_pattern.findall(sentence)
    for i in provider:
        sentence = sentence.replace(i,'')
    return sentence 

In [7]:
#슬래쉬 제공없애기
def remove_slash(sentence):
    slash_pattern = re.compile(r"\/[a-zA-Z]*[가-힣]*=* 제공 *|\/[a-zA-Z]*[가-힣]*=* 기자 *.*") # ~~기자
    slash = slash_pattern.findall(sentence)
    for i in slash:
        sentence = sentence.replace(i,'')
    return sentence 

In [163]:
# 이메일이 제거된 것 확인 할 수 있음.
display(news_train['sentence'].head())
t = news_train['sentence'].apply(remove_email2)
t = t.apply(remove_reporter)
t = t.apply(remove_provider)
t = t.apply(remove_slash)
print(sum(t == ''))
t.head()

0    ha당 조사료 400만원…작물별 차등 지원이성훈 sinawi@hanmail.net전...
1    8억 투입, 고소천사벽화·자산마을에 색채 입혀이성훈 sinawi@hanmail.ne...
2    전남드래곤즈 해맞이 다짐…선수 영입 활발이성훈 sinawi@hanmail.net전남...
3    11~24일, 매실·감·참다래 등 지역특화작목이성훈 sinawi@hanmail.ne...
4    홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항이성훈 sinawi@hanmai...
Name: sentence, dtype: object

0


0    ha당 조사료 400만원…작물별 차등 지원이성훈 전라남도가 쌀 과잉문제를 근본적으로...
1    8억 투입, 고소천사벽화·자산마을에 색채 입혀이성훈 여수시는 원도심 일대에서 추진된...
2    전남드래곤즈 해맞이 다짐…선수 영입 활발이성훈 전남드래곤즈(사장 신승재)는 지난 4...
3    11~24일, 매실·감·참다래 등 지역특화작목이성훈 광양시는 오는 11일부터 24일...
4    홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항이성훈 타이완의 크루즈관광객 4...
Name: sentence, dtype: object

In [ ]:
news_train2 = news_train.copy()
news_train2['sentence'] = t

In [165]:
#기자가 제거된 거 확인할 수 있음
display(news_val['sentence'].head())
t = news_val['sentence'].apply(remove_email2)
t = t.apply(remove_reporter)
t = t.apply(remove_provider)
t = t.apply(remove_slash)
print(sum(t == ''))
t.head()

0    [ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 "넓고, 체...
1    ] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.바른미래당이 8일 연 최...
2    [ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀.지난 3일 한국이 세계 최초로 5...
3    ] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리...
4    [ 임근호 기자 ] "SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를 ...
Name: sentence, dtype: object

0


0    '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 "넓고, 체증 없는 '통신 고속...
1    당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.바른미래당이 8일 연 최고위...
2    8일 서울 올림픽공원 K아트홀.지난 3일 한국이 세계 최초로 5세대(5G) 이동통신...
3    박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리를 ...
4    "SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를 배워라."기업 지배구...
Name: sentence, dtype: object

In [ ]:
news_val2 = news_val.copy()
news_val2['sentence'] = t

In [167]:
display(personal_train['sentence'].head())
t = personal_train['sentence'].apply(remove_email2)
t = t.apply(remove_reporter)
t = t.apply(remove_provider)
t = t.apply(remove_slash)
print(sum(t == ''))
t.head()

0    이명박 대통령이 어제 30대 그룹 총수를 모아놓고 "시대적 요구는 역시 총수가 앞장...
1    이명박 정부의 첫 대통령실장을 지낸 류우익 주중대사가 통일부 장관에 내정되면서 대북...
2    한국은행법 개정안이 우여곡절 끝에 국회 본회의를 통과했다.이 법안은 글로벌 금융위기...
3    이명박 대통령이 어제 30대 그룹 총수를 모아놓고 "시대적 요구는 역시 총수가 앞장...
4    한나라당과 정부가 비정규직 대책을 이르면 다음주 중 내놓을 예정이다.비정규직 임금을...
Name: sentence, dtype: object

0


0    이명박 대통령이 어제 30대 그룹 총수를 모아놓고 "시대적 요구는 역시 총수가 앞장...
1    이명박 정부의 첫 대통령실장을 지낸 류우익 주중대사가 통일부 장관에 내정되면서 대북...
2    한국은행법 개정안이 우여곡절 끝에 국회 본회의를 통과했다.이 법안은 글로벌 금융위기...
3    이명박 대통령이 어제 30대 그룹 총수를 모아놓고 "시대적 요구는 역시 총수가 앞장...
4    한나라당과 정부가 비정규직 대책을 이르면 다음주 중 내놓을 예정이다.비정규직 임금을...
Name: sentence, dtype: object

In [ ]:
personal_train2 = personal_train.copy()
personal_train2['sentence'] = t

In [169]:
display(personal_val['sentence'].head())
t = personal_val['sentence'].apply(remove_email2)
t = t.apply(remove_reporter)
t = t.apply(remove_provider)
t = t.apply(remove_slash)
print(sum(t == ''))
t.head()

0    더불어민주당 이해찬 대표가 30 일 오후 국회에서 기자간담회를 열고 조국 전 법무부...
1    탈원전 정책에서 비롯된 한국전력의 경영 부담이 결국 전기료 인상을 초래하게 됐다.김...
2    북한이 그제 금강산국제관광국 명의로 통일부와 현대아산에 통지문을 보내 금강산관광 시...
3    지난달 한국의 소비자물가상승률이 경제협력개발기구(OECD) 회원국 중 가장 낮았다....
4    김종갑 한국전력 사장이 1 조1 천억원이 넘는 각종 전기료 특례 할인을 모두 폐지하...
Name: sentence, dtype: object

0


0    더불어민주당 이해찬 대표가 30 일 오후 국회에서 기자간담회를 열고 조국 전 법무부...
1    탈원전 정책에서 비롯된 한국전력의 경영 부담이 결국 전기료 인상을 초래하게 됐다.김...
2    북한이 그제 금강산국제관광국 명의로 통일부와 현대아산에 통지문을 보내 금강산관광 시...
3    지난달 한국의 소비자물가상승률이 경제협력개발기구(OECD) 회원국 중 가장 낮았다....
4    김종갑 한국전력 사장이 1 조1 천억원이 넘는 각종 전기료 특례 할인을 모두 폐지하...
Name: sentence, dtype: object

In [ ]:
personal_val2 = personal_val.copy()
personal_val2['sentence'] = t

In [185]:
news_train2.to_feather(f'{DATA_PATH}news_train3.ftr')
news_val2.to_feather(f'{DATA_PATH}news_val3.ftr')
personal_train2.to_feather(f'{DATA_PATH}personal_train3.ftr')
personal_val2.to_feather(f'{DATA_PATH}personal_val3.ftr')

In [286]:
news_train = pd.read_feather(f'{DATA_PATH}news_train3.ftr')
news_val = pd.read_feather(f'{DATA_PATH}news_val3.ftr')
personal_train = pd.read_feather(f'{DATA_PATH}personal_train3.ftr')
personal_val = pd.read_feather(f'{DATA_PATH}personal_val3.ftr')

In [287]:
#뉴스와 사설 데이터 합치기
news_train = news_train[['sentence', 'abstractive']]
news_val = news_val[['sentence', 'abstractive']]
personal_train = personal_train[['sentence', 'abstractive']]
personal_val = personal_val[['sentence', 'abstractive']]

train_data = pd.concat([news_train, personal_train])
val_data = pd.concat([news_val, personal_val])

display(train_data.head())
display(val_data.head())

print(train_data.shape)
print(val_data.shape)

,sentence,abstractive
0,ha당 조사료 400만원…작물별 차등 지원이성훈 전라남도가 쌀 과잉문제를 근본적으로...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...
1,"8억 투입, 고소천사벽화·자산마을에 색채 입혀이성훈 여수시는 원도심 일대에서 추진된...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...
2,전남드래곤즈 해맞이 다짐…선수 영입 활발이성훈 전남드래곤즈(사장 신승재)는 지난 4...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...
3,"11~24일, 매실·감·참다래 등 지역특화작목이성훈 광양시는 오는 11일부터 24일...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작..."
4,"홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항이성훈 타이완의 크루즈관광객 4...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...


,sentence,abstractive
0,"'대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, 체증 없는 '통신 고속...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...
1,당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.바른미래당이 8일 연 최고위...,8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...
2,8일 서울 올림픽공원 K아트홀.지난 3일 한국이 세계 최초로 5세대(5G) 이동통신...,지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...
3,박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리를 ...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...
4,"""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를 배워라.""기업 지배구...",주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...


(300743, 2)
(37130, 2)


In [288]:
#요약문이 비어 있으면 NA로 대체
train_data.loc[train_data.abstractive == '', 'abstractive'] = np.nan
val_data.loc[val_data.abstractive == '', 'abstractive'] = np.nan

In [289]:
print(train_data.isnull().sum().sum(), val_data.isnull().sum().sum())
print(train_data.duplicated().sum(),val_data.duplicated().sum())

8 1
20 2


In [290]:
#NA를 모두 제거
train_data.dropna(inplace = True)
val_data.dropna(inplace = True)

In [291]:
print(sum(train_data.sentence == '') ,sum(val_data.sentence == ''))
print(sum(train_data.abstractive == '') ,sum(val_data.abstractive == ''))

0 0
0 0


In [292]:
print(train_data.shape, val_data.shape)
train_data.drop_duplicates(inplace = True)
val_data.drop_duplicates(inplace = True)
print(train_data.shape, val_data.shape)

(300735, 2) (37129, 2)
(300715, 2) (37127, 2)


In [293]:
# 기존 데이터
news_train = pd.read_feather(f'{DATA_PATH}news_train.ftr')
personal_train = pd.read_feather(f'{DATA_PATH}personal_train.ftr')
news_val = pd.read_feather(f'{DATA_PATH}news_val.ftr')
personal_val = pd.read_feather(f'{DATA_PATH}personal_val.ftr')

news_train = news_train[['sentence', 'abstractive']]
news_val = news_val[['sentence', 'abstractive']]
personal_train = personal_train[['sentence', 'abstractive']]
personal_val = personal_val[['sentence', 'abstractive']]

train_data_original = pd.concat([news_train, personal_train])
val_data_original = pd.concat([news_val, personal_val])

display(train_data_original.head())
display(val_data_original.head())

print(train_data_original.shape)
print(val_data_original.shape)

,sentence,abstractive
0,ha당 조사료 400만원…작물별 차등 지원이성훈 sinawi@hanmail.net전...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...
1,"8억 투입, 고소천사벽화·자산마을에 색채 입혀이성훈 sinawi@hanmail.ne...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...
2,전남드래곤즈 해맞이 다짐…선수 영입 활발이성훈 sinawi@hanmail.net전남...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...
3,"11~24일, 매실·감·참다래 등 지역특화작목이성훈 sinawi@hanmail.ne...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작..."
4,"홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항이성훈 sinawi@hanmai...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...


,sentence,abstractive
0,"[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, 체...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...
1,] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.바른미래당이 8일 연 최...,8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...
2,[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀.지난 3일 한국이 세계 최초로 5...,지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...
3,] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...
4,"[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를 ...",주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...


(300743, 2)
(37130, 2)


In [294]:
train_data_original.loc[train_data_original.abstractive == '', 'abstractive'] = np.nan
val_data_original.loc[val_data_original.abstractive == '', 'abstractive'] = np.nan

In [295]:
print(train_data_original.isnull().sum().sum(), val_data_original.isnull().sum().sum())
train_data_original.dropna(inplace = True)
val_data_original.dropna(inplace = True)

print(train_data_original.isnull().sum().sum(), val_data_original.isnull().sum().sum())
print(train_data_original.shape, val_data_original.shape)

8 1
0 0
(300735, 2) (37129, 2)


In [297]:
#orignal + 전처리 데이터
train_data = pd.concat([train_data_original, train_data])
val_data = pd.concat([val_data_original, val_data])

train_data.drop_duplicates(inplace = True)
val_data.drop_duplicates(inplace = True)

print(train_data.shape)
print(val_data.shape)

(403659, 2)
(39560, 2)


In [281]:
# KoBART 예제에서 제공하는 데이터
train_ko = pd.read_csv('...../train.tsv', sep = '\t')
test_ko = pd.read_csv('..../test.tsv', sep = '\t')
train_ko.head()

,news,summary
0,일제 강제징용피해자들이 지난 2월 발의된 ‘근로정신대 피해자 지원법률안’의 조속한 ...,13일 광주시의회 시민 소통실에서 시민단체 '근로정신대 할머니와 함께 하는 시민모임...
1,문재인 대통령이 이르면 9일 중폭 개각을 단행할 것으로 보인다. 여권 관계자는 8일...,"문재인 대통령이 이르면 9일 장관 3~4명, 장관급 3명을 교체하는 개각을 단행하고..."
2,전남 어촌마을 정주여건 개선 ‘탄력’ 해수부 공모서 고흥 득량도·강진 신전 등 4개...,"해양수산부 어촌종합개발 거점 개발사업에 전남 고흥 득량도, 강진 신전, 완도 충동,..."
3,여자친구 알몸을 무단으로 촬영해 인터넷 성인 카페에 유포한 남성이 재판에 넘겨졌다....,피해자 A씨는 가해자인 B씨의 엄벌을 촉구하며 국민청원에 피해사실을 공개하였고 결국...
4,대구시의회 배지숙 의장이 개회를 선언하고 있다. 대구시의회(의장 배지숙)는 14일부...,대구시의회가 14일 제1차 본회의를 시작으로 제265회 임시회를 개최하고 ‘대구시시...


In [282]:
print(train_ko.isnull().sum().sum(), test_ko.isnull().sum().sum())
print(train_ko.duplicated().sum(), test_ko.duplicated().sum())
print(train_ko.shape, test_ko.shape)

0 0
0 0
(34242, 2) (8561, 2)


In [298]:
train_data.columns = train_ko.columns
val_data.columns = test_ko.columns

#기존 + 전처리 + KoBART 예제 데이터
train = pd.concat([train_data, train_ko])
test = pd.concat([val_data, test_ko])

train.reset_index(inplace = True, drop = True)
test.reset_index(inplace = True, drop = True)

display(train.head())
display(test.head())
print(train.shape, test.shape)

,news,summary
0,ha당 조사료 400만원…작물별 차등 지원이성훈 sinawi@hanmail.net전...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...
1,"8억 투입, 고소천사벽화·자산마을에 색채 입혀이성훈 sinawi@hanmail.ne...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...
2,전남드래곤즈 해맞이 다짐…선수 영입 활발이성훈 sinawi@hanmail.net전남...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...
3,"11~24일, 매실·감·참다래 등 지역특화작목이성훈 sinawi@hanmail.ne...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작..."
4,"홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항이성훈 sinawi@hanmai...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...


,news,summary
0,"[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, 체...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...
1,] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.바른미래당이 8일 연 최...,8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...
2,[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀.지난 3일 한국이 세계 최초로 5...,지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...
3,] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...
4,"[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를 ...",주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...


(437901, 2) (48121, 2)


In [299]:
train.to_csv(f'{DATA_PATH}train.tsv', header=True, sep='\t', index=False)
test.to_csv(f'{DATA_PATH}test.tsv', header=True, sep='\t', index=False)